In [ ]:
!pip install opencv-python
!pip install imutils
!pip install numpy
!pip install easyocr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.9 MB 25.1 MB/s 
     |████████████████████████████████| 145 kB 65.4 MB/s 
     |█████████████████████████▋      | 38.0 MB 1.3 MB/s eta 0:00:08

In [ ]:
import cv2
import imutils
import numpy as np
import easyocr

import datetime
from time import time

def write(message):
    f = open("data.txt", "r")
    result = f.read().find(message)
    f.close()
    if result ==-1:
        time = datetime.datetime.now()
        f = open("data.txt","a")
        f.write(message +" ")
        f.write(str(time)+"\n")
        f.close

def rec(img):
    #Filtreleme islemi
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.bilateralFilter(gray, 11, 17, 17)
    edged = cv2.Canny(gray, 30, 200)
    try:
        cnts = cv2.findContours(edged.copy(), cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:10]
        screenCnt = None
        for c in cnts:
            peri = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.017 * peri, True)
            if len(approx) == 4:
                screenCnt = approx
                break
        if screenCnt is None:
            detected = 0
        else:
            detected = 1

        if detected == 1:
            cv2.drawContours(img, [screenCnt], -1, (0, 255, 0), 3)
        #Maskeleme islemi
        mask = np.zeros(gray.shape, np.uint8)
        new_image = cv2.drawContours(mask, [screenCnt], 0, 255, -1,)
        new_image = cv2.bitwise_and(img, img, mask=mask)
        #Kırma islemi
        (x, y) = np.where(mask == 255)
        (topx, topy) = (np.min(x), np.min(y))
        (bottomx, bottomy) = (np.max(x), np.max(y))
        Cropped = gray[topx:bottomx + 1, topy:bottomy + 1]
        #EasyOCR islemi
        reader = easyocr.Reader(['en'])
        text = reader.readtext(Cropped)
        text = text[0][1]
        text = text.replace(" ", "")
        text = text.upper()
        sonuc = 0
        if(len(text)>=7):
            if(int(text[0]) and int(text[1])):
                if(str(text[4])):
                    if(int(text[5:])):
                        sonuc=1
                else:
                    if(int(text[4:])):
                        sonuc=1
        if(sonuc==1):
            save.write(text)
            print(text)
    except Exception:
        pass
    return img


import imageProcessing as imgprocess

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    img = imgprocess.rec(frame)
    cv2.imshow('Screen', img)
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()